In [1]:
# Import libs
import requests
from pandas.core.common import flatten
import datetime as dt
from datetime import date
import pandas as pd


In [2]:
interest_cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

# I. Get Cities Coordinates

In [3]:
keys = ['lat', 'lon'] #Select what we're interested in
city_coordinates = []
for city in interest_cities: 
    new_data = [city] #empty new data list to have one for that city
    r = requests.get("https://nominatim.openstreetmap.org/search/{}?format=json&addressdetails=1&limit=1&polygon_svg=1>" \
                     .format(city)) #Make the request
    new_data.append([r.json()[0].get(key) for key in keys]) #Get latitude and flatiing
    new_data = list(flatten(new_data)) # Flatten list otherwise nested list
    city_coordinates.append(new_data) # Create nested list with the other cities

print(len(city_coordinates)==len(interest_cities)) #sense check -- should return true
print(city_coordinates) #check everything looks fine

True
[['Mont Saint Michel', '48.6359541', '-1.511459954959514'], ['St Malo', '48.649518', '-2.0260409'], ['Bayeux', '49.2764624', '-0.7024738'], ['Le Havre', '49.4938975', '0.1079732'], ['Rouen', '49.4404591', '1.0939658'], ['Paris', '48.8588897', '2.3200410217200766'], ['Amiens', '49.8941708', '2.2956951'], ['Lille', '50.6365654', '3.0635282'], ['Strasbourg', '48.584614', '7.7507127'], ['Chateau du Haut Koenigsbourg', '48.249489800000006', '7.34429620253195'], ['Colmar', '48.0777517', '7.3579641'], ['Eguisheim', '48.0447968', '7.3079618'], ['Besancon', '47.2380222', '6.0243622'], ['Dijon', '47.3215806', '5.0414701'], ['Annecy', '45.8992348', '6.1288847'], ['Grenoble', '45.1875602', '5.7357819'], ['Lyon', '45.7578137', '4.8320114'], ['Gorges du Verdon', '43.7496562', '6.3285616'], ['Bormes les Mimosas', '43.1572172', '6.329253867921363'], ['Cassis', '43.2140359', '5.5396318'], ['Marseille', '43.2961743', '5.3699525'], ['Aix en Provence', '43.5298424', '5.4474738'], ['Avignon', '43.9492

# II. Get Weather Data

In [8]:
## Naming some variables to be used in for loop later

    ## For the requests
api_key = "7f6b9287bfa4d90843a0002a16be3b43"
units = "metric" #Get temperature in celsius


    ## For the cleaning up 
#Because one of the element is more nested than the other, we need to do slightly more work on unpacking
weather_keys = ["dt",
                "feels_like",# to unpack further, + taking that one instead of temperature
               "clouds",
                'pop',
                'rain',
                'uvi'
               ]

weather_data = [] # This will become a list of dict with the same keys to then be transformed to pandas

#Get cleaner keys to then pass to pandas from dict once already unpacked
weather_clean_keys = ['city','date','day_feels_like_temp','cloudiness_percent',"prob_rain","rain","max_uv"]


## Let's get all the data and clean it all up 

for city_list in city_coordinates:
    
    ## Requesting the data
    
    #Excluded current, minutely & hourly data as we only need daily & potentially alerts
    r = requests.get("https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&units={}&exclude=current,minutely,hourly&appid={}"\
                 .format(city_list[1],city_list[2],units,api_key)) #Each city list has first the city name, then latitude then longitude
    
    
    ## Cleaning up the output
    
    #Now for each city, we want to get clean data in the form of a clean list of dict with always the same keys
    # This list of dict will then be passed to create a pandas dataframe
    for date in r.json()['daily']:
        new_weather_data = [date.get(key) for key in weather_keys] ## need to unpack futher the day 
        new_weather_data = [i if type(i) != dict else i['day'] for i in new_weather_data]
        new_weather_data.insert(0,city_list[0])
        new_weather_data = dict(zip(weather_clean_keys,new_weather_data)) #transform list to dict with clean keys
        weather_data.append(new_weather_data)
    
    

weather_df = pd.DataFrame(weather_data)
weather_df.head(5)

,city,date,day_feels_like_temp,cloudiness_percent,prob_rain,rain,max_uv
0,Mont Saint Michel,1644062400,6.96,12,0.37,0.13,1.22
1,Mont Saint Michel,1644148800,10.44,100,0.78,1.86,0.63
2,Mont Saint Michel,1644235200,8.18,20,0.56,NaN,0.88
3,Mont Saint Michel,1644321600,9.45,16,0.00,NaN,1.44
4,Mont Saint Michel,1644408000,8.87,2,0.00,NaN,1.50


In [14]:
# Clean up DataFrame further
#weather_df['test'] = pd.Timestamp(weather_df['date'],unit='s')
weather_df['test'] = weather_df.date.assign(Product= lambda x: pd.Timestamp(x,unit='s'))
weather_df.head()

AttributeError: 'Series' object has no attribute 'assign'

## III. Get Hotel Data

In [12]:
pd.Timestamp(1644062400,unit='s')  ## To put on entire colymn, maybe lambda not sure

Timestamp('2022-02-05 12:00:00')